In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import spacy
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import classification_report as report

In [2]:
lab = pd.read_csv("patients_no_text_sepsis_labels.csv")
text = pd.read_csv("noteevents_preprocessed_text_granular_label.csv")

In [4]:
text.dropna(subset=['clean_text'], inplace=True)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text

nlp = spacy.load("en_core_web_sm", disable=['tagger', 'ner'])

In [5]:
IDs = pd.read_csv("IDs.csv")
train_IDs = np.unique(IDs["train_ids"][~np.isnan(IDs["train_ids"])])
test_IDs = np.unique(IDs["test_ids"][~np.isnan(IDs["test_ids"])])
val_IDs = np.unique(IDs["val_ids"][~np.isnan(IDs["val_ids"])])
print(np.intersect1d(train_IDs,test_IDs))
print(np.intersect1d(train_IDs,val_IDs))
print(np.intersect1d(test_IDs,val_IDs))

[]
[]
[]


In [41]:
temp_ = lab.copy()
temp_["granular_label"] = np.where(((temp_["m:charttime"] >= temp_["m:presumed_onset"]) & (temp_["m:presumed_onset"] > 0.0)), 1, 0)
rows = temp_.drop(columns=["sepsis_label", "m:charttime", "traj", "r:reward", "a:action", "o:output_4hourly", "o:output_total", "o:PaO2_FiO2", "Unnamed: 0", "Unnamed: 0.1", "m:presumed_onset", "o:SOFA", "o:SIRS", "o:input_4hourly", "o:input_total", "o:GCS", "o:PTT", "o:DiaBP", "o:MeanBP", "o:SpO2", "o:cumulated_balance", "o:Creatinine", "o:Platelets_count", "o:re_admission", "o:mechvent", "o:max_dose_vaso"])
r = rows.copy()
r = r.loc[r["step"]<=4]
r.drop(columns="step", inplace=True)
X_train = r.loc[r["m:icustayid"].isin(train_IDs)]
y_train = X_train["granular_label"]
X_test = r.loc[r["m:icustayid"].isin(test_IDs)]
y_test = X_test["granular_label"]
X_val = r.loc[r["m:icustayid"].isin(val_IDs)]
y_val = X_val["granular_label"]


X_train.drop(columns=["granular_label", "m:icustayid"], inplace=True)
X_test.drop(columns=["granular_label", "m:icustayid"], inplace=True)
X_val.drop(columns=["granular_label", "m:icustayid"], inplace=True)

/var/folders/rm/v29286yx225c2ktvpkpwfr5r0000gq/T/ipykernel_29407/106022131.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(columns=["granular_label", "m:icustayid"], inplace=True)
/var/folders/rm/v29286yx225c2ktvpkpwfr5r0000gq/T/ipykernel_29407/106022131.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop(columns=["granular_label", "m:icustayid"], inplace=True)
/var/folders/rm/v29286yx225c2ktvpkpwfr5r0000gq/T/ipykernel_29407/106022131.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyda

In [49]:
X_train

,o:Arterial_BE,o:Arterial_lactate,o:Arterial_pH,o:BUN,o:Calcium,o:Chloride,o:FiO2_1,o:Glucose,o:HCO3,o:HR,...,o:Sodium,o:SysBP,o:Temp_C,o:Total_bili,o:WBC_count,o:Weight_kg,o:age,o:gender,o:paCO2,o:paO2
431,0.145024,-0.065495,-0.193226,0.874827,0.592544,-0.275503,3.020434,-0.431229,-0.135694,0.540450,...,-0.110016,0.852508,-0.011446,-0.327560,1.260355,-0.450723,-0.056854,-0.5,0.342991,1.257898
432,0.145024,-0.065495,-0.193226,0.874827,0.592544,-0.275503,3.020434,-0.431229,-0.135694,0.540450,...,-0.110016,0.852508,0.004119,-0.327560,1.260355,2.673019,-0.056854,-0.5,0.342991,1.257898
433,0.145024,-0.065495,-0.193226,0.874827,0.592544,-0.275503,3.020434,-0.431229,-0.135694,0.715875,...,-0.110016,-0.690851,0.004119,0.434184,1.260355,2.673019,-0.056854,-0.5,0.342991,1.257898
434,0.145024,-0.065495,-0.193226,0.874827,0.592544,-0.275503,3.020434,-0.431229,-0.135694,0.598925,...,-0.110016,-1.439146,-0.011446,-0.327560,1.260355,2.673019,-0.056854,-0.5,0.342991,1.257898
435,0.145024,-0.065495,-0.193226,0.874827,0.592544,-0.275503,3.020434,-0.431229,-0.135694,0.423499,...,-0.110016,-0.410240,0.004119,-0.327560,1.260355,2.673019,-0.056854,-0.5,0.342991,1.257898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991820,-1.185859,1.668771,-0.040313,0.242901,0.367001,0.728841,-0.340459,0.872804,-0.644026,-1.358297,...,-0.494927,0.255416,0.208684,-0.891924,0.197851,0.254079,-0.160118,-0.5,-1.125021,0.212572
1991821,-0.278722,1.367127,0.321988,-0.303126,0.910616,-0.525168,3.481176,0.308343,-0.644026,-1.027079,...,-0.376929,0.278966,0.401694,0.025693,-0.793800,1.608256,-0.160118,-0.5,-0.705730,-1.016493
1991822,-0.278722,1.065483,0.321988,-0.303126,0.910616,-0.525168,3.481176,0.308343,-0.644026,-0.906637,...,-0.376929,0.161219,0.291403,0.025693,-0.793800,1.608256,-0.160118,-0.5,-0.705730,-1.016493
1991823,-0.278722,1.367127,0.321988,-0.303126,0.910616,-0.525168,3.481176,0.308343,-0.644026,-0.966858,...,-0.376929,0.161219,0.291403,0.025693,-0.793800,1.608256,-0.160118,-0.5,-0.705730,-1.016493


In [43]:
print(f"Shape of X_train: {len(X_train)}")
print(f"Shape of X_test: {len(X_test)}")
print("Class Balance in Training Data:", Counter(y_train).items())
print("Class Balance in Test Data:", Counter(y_test).items())

Shape of X_train: 114380
Shape of X_test: 25420
Class Balance in Training Data: dict_items([(0, 106402), (1, 7978)])
Class Balance in Test Data: dict_items([(0, 23679), (1, 1741)])


In [18]:
X_train.columns

Index(['o:Arterial_BE', 'o:Arterial_lactate', 'o:Arterial_pH', 'o:BUN',
       'o:Calcium', 'o:Chloride', 'o:FiO2_1', 'o:Glucose', 'o:HCO3', 'o:HR',
       'o:Hb', 'o:INR', 'o:Magnesium', 'o:PT', 'o:Potassium', 'o:RR', 'o:SGOT',
       'o:SGPT', 'o:Shock_Index', 'o:Sodium', 'o:SysBP', 'o:Temp_C',
       'o:Total_bili', 'o:WBC_count', 'o:Weight_kg', 'o:age', 'o:gender',
       'o:paCO2', 'o:paO2'],
      dtype='object')

In [44]:
clf = make_pipeline(StandardScaler(),RandomForestClassifier(n_jobs=-1, verbose=1))


In [45]:
clf.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    5.1s finished


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(n_jobs=-1, verbose=1))])

In [48]:
print("Score:", clf.score(X_val, y_val))
pr_forest = clf.predict(X_test)
print(report(y_true=y_test, y_pred=pr_forest))

Score: 0.9294256490952006
              precision    recall  f1-score   support

           0       0.93      1.00      0.96     23679
           1       0.06      0.00      0.00      1741

    accuracy                           0.93     25420
   macro avg       0.50      0.50      0.48     25420
weighted avg       0.87      0.93      0.90     25420



[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
